# The purpose of this notebook is for experimenting with code snippets

Monte Carlo in MC

In [1]:
from adaptive_time.environment import MountainCar
from adaptive_time.features import Fourier_Features
import numpy as np
from adaptive_time.utils import argmax

In [2]:
phi = Fourier_Features()
phi.init_fourier_features(2,3)
phi.init_state_normalizers(np.array([0.6,0.07]), np.array([-1.2,-0.07]))

phi.get_fourier_feature([-0.5,0.0])

array([ 1.00000000e+00,  6.12323400e-17, -1.00000000e+00, -1.83697020e-16,
        8.66025404e-01, -5.00000000e-01, -8.66025404e-01,  5.00000000e-01,
        5.00000000e-01, -8.66025404e-01, -5.00000000e-01,  8.66025404e-01,
       -6.04901475e-16, -1.00000000e+00,  7.04481400e-16,  1.00000000e+00])

In [17]:



def foo(xs,tol):
    c = int(len(xs) / 2)
    f = lambda xs: len(xs) * ( xs[0] + xs[-1]) / 2 if len(xs) else 0
    if abs(f(xs) - (r := f(xs[:c]) + f(xs[c:]))) < tol: 
        return 1, r
    else: 
        x, fa = foo(xs[:c], tol / 2)
        y, fb = foo(xs[c:], tol / 2)
        return x + y, fa + fb


def generate_trajectory(env, weights, episode, num_actions = 3):
    trajectory = []
    s = env.reset(episode)
    position, velocity = s
    done = False
    while not done:
        state_feature = phi.get_fourier_feature(s)
        action_value = np.zeros(num_actions) #For mountain car
        for action in range(num_actions):
            action_value[action] = np.inner(state_feature, weights[action])
        a = argmax(action_value)
        r, s_, _, done = env.step(a)
        trajectory.append([s, a, r, s_])
        s = s_
    return trajectory

    

def monte_carlo(env, phi, weights, targets, feature_matrix, num_actions, episode, gamma = 0.99):
    
    rewards = []
    trajectory = generate_trajectory(env, weights, episode, num_actions)
    N = len(trajectory)
    G = 0
    for t in range(N-1,-1,-1):
        state, action, reward, state_ = trajectory[t]
        G = G + reward
        x = phi.get_fourier_feature(state) 

        #setup least-squares via Sherman Morrison
        feature_matrix[action] = feature_matrix[action] + np.outer(x,x)
        targets[action] = targets[action] + G * x
        weights[action] = np.linalg.solve(feature_matrix[action], targets[action])

        rewards.append(reward)
    return weights, targets, feature_matrix, sum(rewards)



num_actions = 3
num_episodes = 200
env = MountainCar(es=100)
phi = Fourier_Features()
phi.init_fourier_features(2,2)
phi.init_state_normalizers(np.array([0.6,0.07]), np.array([-1.2,-0.07]))
d = len(phi.get_fourier_feature([0.0,0.0]))
weights = np.zeros((num_actions, d)) 
targets = np.zeros((num_actions, d))
feature_matrix = np.zeros((num_actions, d, d))
for action in range(num_actions):
    feature_matrix[action] = np.identity(d)

for episode in range(num_episodes):
    weights, targets, feature_matrix, returns = monte_carlo(env, phi, weights, targets, feature_matrix, num_actions, episode)
    print(episode, returns)



    

0 -200.0
1 -200.0
2 -200.0
3 -200.0
4 -200.0
5 -200.0
6 -200.0
7 -200.0
8 -28.0
9 -200.0
10 -14.0
11 -200.0
12 -200.0
13 -9.0
14 -200.0
15 -200.0
16 -200.0
17 -36.0
18 -200.0
19 -18.0
20 -200.0
21 -200.0
22 -39.0
23 -200.0
24 -200.0
25 -200.0
26 -14.0
27 -200.0
28 -200.0
29 -13.0
30 -200.0
31 -200.0
32 -200.0
33 -11.0
34 -200.0
35 -200.0
36 -200.0
37 -200.0
38 -22.0
39 -200.0
40 -32.0
41 -200.0
42 -33.0
43 -200.0
44 -200.0
45 -200.0
46 -200.0
47 -23.0
48 -5.0
49 -23.0
50 -200.0
51 -200.0
52 -200.0
53 -200.0
54 -200.0
55 -36.0
56 -200.0
57 -200.0
58 -200.0
59 -200.0
60 -200.0
61 -200.0
62 -200.0
63 -26.0
64 -200.0
65 -200.0
66 -200.0
67 -30.0
68 -200.0
69 -19.0
70 -30.0
71 -8.0
72 -38.0
73 -200.0
74 -200.0
75 -200.0
76 -200.0
77 -200.0
78 -200.0
79 -200.0
80 -200.0
81 -200.0
82 -200.0
83 -9.0
84 -200.0
85 -200.0
86 -200.0
87 -200.0
88 -200.0
89 -200.0
90 -22.0
91 -16.0
92 -200.0
93 -200.0
94 -200.0
95 -33.0
96 -200.0
97 -200.0
98 -200.0
99 -200.0
100 -200.0
101 -200.0
102 -200.0
103 -20

# Quadrature

In [10]:
feature_matrix

array([[[             nan,              nan,              nan,
                      nan,              nan,              nan,
                      nan,              nan,              nan,
                      nan,              nan,              nan,
                      nan,              nan,              nan,
                      nan],
        [             nan,              nan,              nan,
                      nan,              nan,              nan,
                      nan,              nan,              nan,
                      nan,              nan,              nan,
                      nan,              nan,              nan,
                      nan],
        [             nan,              nan,              nan,
                      nan,              nan,              nan,
                      nan,              nan,              nan,
                      nan,              nan,              nan,
                      nan,              nan,              nan,

In [ ]:
def trapeziod_rule(a,b):
    return (b-a) * (function(a) + function(b)) / 2.0

def integral_rule(a,b):
    return trapeziod_rule(a,b)

def function(x):
    return x**5

def adaptive_quadrature(a0, b0, tol0):
    sums = 0.0
    n = 1
    a = np.zeros(100000)
    b = np.zeros(100000)
    tol = np.zeros(100000)
    app = np.zeros(100000)
    iters = 0
    
    a[1] = a0
    b[1] = b0
    tol[1] = tol0
    app[1] = integral_rule(a0,b0)
    
    while n > 0:
        iters += 1
        c = (a[n] + b[n]) / 2
        oldapp = app[n]
        app[n] = integral_rule(a[n], c)
        app[n+1] = integral_rule(c, b[n])
        
        if np.abs(oldapp - (app[n]+app[n+1])) < 3 * tol[n]:
            sums = sums + app[n] + app[n+1] #success
            n = n - 1 #done with interval
            
        else:    #divide into two intervals
            b[n+1] = b[n] #setup new intervals
            b[n] = c  #setup new intervals
            a[n+1] = c #setup new intervals
            tol[n] = tol[n] / 2
            tol[n+1] = tol[n]
            n = n + 1
    return sums,iters
        
        

In [ ]:
b = 15
truth = b**5/5

quad,iters = adaptive_quadrature(0, b, 0.00005)

h = iters
x = np.linspace(0, b, num=h)
y = function(x)
trap = np.trapz(y,x)

In [ ]:
print(np.abs(truth - quad))
print(np.abs(truth - trap))

print(iters - h)

1746562.5000270708
1746562.5000363498
0


In [16]:
import math
def adaptive_sum(traj, tol0=10**(-2)):
    steps = {}
    N = len(traj)
    app = np.zeros(N+1000)
    tol = np.zeros(N+1000)
    N_begin = np.ones(N+1000,dtype=np.int8) * int(-1)
    N_end = np.ones(N+1000,dtype=np.int8) * int(-1)

    sums = 0.0


    steps[N_begin[0]] = 1
    steps[N_end[0]] = 1

    n = 0
    N_begin[0] = 0
    N_end[0] = N - 1
    tol[0] = tol0
    app[0] = (N_end[0] - N_begin[0] + 1) / (2) * ( traj[N_begin[0]] + traj[N_end[0]] )
    iters = 0
    while n > -1: 
        iters += 1
        if (N_end[n] - N_begin[n] + 1) % 2 == 0:
            N_split = int( (N_end[n] - N_begin[n] + 1) / 2 )

        else:
            N_split = math.ceil((N_end[n] - N_begin[n] + 1) / 2)

        old_app = app[n]
        app[n] = (N_split - N_begin[n]) / (2) * ( traj[N_split - 1] + traj[N_begin[n]] )
        app[n+1] = (N_end[n] - N_split + 1) / (2) * ( traj[N_split] + traj[N_end[n]] )
        
        print(iters, traj[N_begin[n]], traj[N_split - 1], traj[N_split], traj[N_end[n]])

        if abs(old_app - (app[n] + app[n+1])) < 3 * tol[n]:
            sums = sums + app[n] + app[n+1]
            n = n - 1

        else:
            N_end[n+1] = N_end[n]
            N_end[n] = N_split - 1
            N_begin[n+1] = N_split 
            tol[n] = tol[n] / 2
            tol[n+1] = tol[n]
            n = n + 1
            
    return sums


# Quadrature for Discrete Integrals (i.e. sums)

In [23]:
def foo(xs,tol):
    c = int(len(xs) / 2)
    f = lambda xs: len(xs) * ( xs[0] + xs[-1]) / 2 if len(xs) else 0
    if abs(f(xs) - (r := f(xs[:c]) + f(xs[c:]))) < tol: 
        return 1, r
    else: 
        x, fa = foo(xs[:c], tol / 2)
        y, fb = foo(xs[c:], tol / 2)
        return x + y, fa + fb
    

In [24]:
for _ in range(1000):
    traj = np.zeros(20000)
    traj[0:np.random.randint(20000)] = -1
    pivots = []
    calls, approx_sum, pivots = foo(traj, 0.01)
    print(calls, c, approx_sum - sum(traj))

5000
1250
625
39
19
9
5
2
1
1
78
156
312
2500


ValueError: not enough values to unpack (expected 3, got 2)

In [7]:
s = np.ones((10,1))

In [9]:
s.flatten()

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])